# TIME SERIES M

## Chuẩn bị Dữ liệu và Xác định Mô hình (Model Identification)
1. Chọn mã Stock (vd: QQQ)
2. Tải về giá đóng cửa (Close) hàng ngày từ 2015-2025

3. Kiểm tra tính dừng (d)
- Chạy kiểm định ADF (Augmented Dickey-Fuller) trên chuỗi giá (Price). Kết quả (p-value > 0.05) sẽ xác nhận là không dừng.
- Lấy lợi nhuận log (log returns): $log\_return = \log(Price_t) - \log(Price_{t-1}). Tức là d=1.
- Chạy lại kiểm định ADF trên chuỗi log_return. Nếu p-value < 0.05, chuỗi này là dừng. Khi đó chúng ta đã xác định được tham số d = 1.

4. Xác định Tham số (p, q)

## Backtesting với Cửa sổ Trượt (Rolling Window)
1. Thiết lập cửa sổ: Chọn một cửa sổ huấn luyện (lookback period), ví dụ: 100 ngày.
2. Vòng lặp Hàng ngày (Daily Loop): Bắt đầu từ ngày thứ 101 của dữ liệu:
- Huấn luyện: Lấy dữ liệu 100 ngày vừa qua (từ $t-100$ đến $t-1$).
- Fit Mô hình: Huấn luyện (fit) mô hình ARIMA(1, 1, 1) trên 100 ngày dữ liệu giá này.
- Dự báo: Yêu cầu mô hình dự báo giá trị cho ngày $t$ (hôm nay).
3. Quy tắc giao dịch:Lấy giá trị dự báo ($Forecast_t$) và giá trị thực tế của ngày hôm qua ($Price_{t-1}$).
- Nếu $Forecast_t > Price_{t-1}$ (Dự báo Tăng): Mở vị thế Long (Mua) QQQ vào đầu ngày. Đóng vị thế vào cuối ngày.
- Nếu $Forecast_t < Price_{t-1}$ (Dự báo Giảm): Mở vị thế Short (Bán) QQQ vào đầu ngày. Đóng vị thế vào cuối ngày.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings

# Bỏ qua các cảnh báo hội tụ từ statsmodels
warnings.filterwarnings('ignore')

print("Bắt đầu thực hiện Chiến lược Dự báo ARIMA...")

# --- Giai đoạn 1: Tải và Chuẩn bị Dữ liệu ---
print("Giai đoạn 1: Đang tải và xử lý dữ liệu...")

ticker = 'QQQ' # Chỉ dự báo trên 1 tài sản
start_str = '2015-01-01'
end_str = '2023-12-31'

data_raw = yf.download(ticker, start=start_str, end=end_str)
prices = data_raw['Close'].ffill()
log_returns = np.log(prices / prices.shift(1)).dropna()

# --- Giai đoạn 2: Kiểm định Tính dừng ---
print("Giai đoạn 2: Đang kiểm định tính dừng...")

adf_test_prices = adfuller(prices)
print(f'ADF Test (p-value) trên Giá: {adf_test_prices[1]:.4f}')
adf_test_returns = adfuller(log_returns)
print(f'ADF Test (p-value) trên Lợi nhuận Log: {adf_test_returns[1]:.4f}')

if adf_test_returns[1] < 0.05:
    print("-> Lợi nhuận Log là DỪNG (Stationary). Khẳng định d=1 là phù hợp.")
else:
    print("-> Lợi nhuận Log KHÔNG DỪNG. Cần xem xét d=2.")

# --- Giai đoạn 3: Backtest Cửa sổ Trượt (Rolling Window) ---
print("Giai đoạn 3: Bắt đầu vòng lặp backtest... (Việc này sẽ mất vài phút)")

order = (1, 1, 1) # (p, d, q)
lookback_window = 100 # Huấn luyện 100 ngày
refit_every = 21 # Huấn luyện lại 1 tháng/lần

predictions = pd.Series(index=prices.index, dtype=float)
signals = pd.Series(index=prices.index, dtype=float)
model_fit = None

for t in range(lookback_window, len(prices)):

    if (t - lookback_window) % refit_every == 0 or model_fit is None:
        train_data = prices.iloc[t-lookback_window : t]

        try:
            model = ARIMA(train_data, order=order)
            model_fit = model.fit()
        except Exception as e:
            pass

    if model_fit:
        try:
            forecast = model_fit.forecast(steps=1).iloc[0]
            predictions.iloc[t] = forecast

            if forecast > prices.iloc[t-1]:
                signals.iloc[t] = 1 # Dự báo Tăng -> Mua
            else:
                signals.iloc[t] = -1 # Dự báo Giảm -> Bán
        except Exception as e:
            signals.iloc[t] = 0
            pass

print("Backtest hoàn tất.")

# --- Giai đoạn 4: Đánh giá Hiệu suất ---
print("Giai đoạn 4: Đang đánh giá hiệu suất...")

strategy_log_returns = log_returns * signals.shift(1)
strategy_log_returns = strategy_log_returns.dropna()
benchmark_log_returns = log_returns.loc[strategy_log_returns.index]

cumulative_strategy = np.exp(strategy_log_returns.cumsum())
cumulative_benchmark = np.exp(benchmark_log_returns.cumsum())

# --- Giai đoạn 5: Vẽ biểu đồ ---
print("Hoàn tất! Đang tạo biểu đồ...")

plt.figure(figsize=(12, 7))
cumulative_strategy.plot(label='Chiến lược ARIMA(1,1,1)')
cumulative_benchmark.plot(label='Buy & Hold QQQ', linestyle='--')
plt.title('Hiệu suất Chiến lược ARIMA (2015 - 2023)')
plt.ylabel('Lợi nhuận Cộng dồn (Log scale)')
plt.xlabel('Ngày')
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.grid(True, which="both", ls="--", linewidth=0.5)

plt.show()

print("Chương trình kết thúc.")